# 가설1 : 29cm를 많이 가입하는 세대는 M 세대지만, 이용 시간은 Z가 많을 것이다.  

> - 지표마다 전체 코드를 첫번째 셀에 모두 적은 후, 하나씩 실행해서 결과 도출했습니다. 

In [1]:
import pandas as pd
import numpy as np

# 통합 파일 불러오기
df = pd.read_csv("mz_final.csv", encoding = 'utf-8')

## 가입자수

In [2]:
# user_no을 기준으로 중복 제거 
df_user = df.drop_duplicates(['user_no'])

# M/Z 가입자수 비교
df_user['user_no'].groupby(df_user['generation']).count()

generation
M    143873
Z     61944
Name: user_no, dtype: int64

### 전체 MZ 가입자 수에 대한 M,Z 각각의 비율 구하기

In [ ]:
# MZ 가입자수 총합
total = df_user['generation'].value_counts().sum()
total # 205817

# M 가입자 수 총합
total_m = df_user[df_user['generation'] == "M"]['generation'].count()
total_m # 143873

# Z 가입자 수 총합
total_z = df_user[df_user['generation'] == "Z"]['generation'].count()
total_z # 61944

# total_m/total = 전체 MZ에 대한 M의 비율
total_m / total # 0.6990336075251316

# total_z/total = 전체 MZ에 대한 Z의 비율
total_z/total # 0.30096639247486845

In [3]:
# MZ 가입자수 총합
total = df_user['generation'].value_counts().sum()
total

205817

In [4]:
# M 가입자 수 총합
total_m = df_user[df_user['generation'] == "M"]['generation'].count()
total_m

143873

In [5]:
# Z 가입자 수 총합
total_z = df_user[df_user['generation'] == "Z"]['generation'].count()
total_z 

61944

In [7]:
# total_m/total = 전체 MZ에 대한 M의 비율
total_m / total

0.6990336075251316

In [8]:
# total_z/total = 전체 MZ에 대한 Z의 비율
total_z/total

0.30096639247486845

## 체류시간

> - M세대 구하는 코드만 올린 것으로 "Z"로 바꿔서 Z세대 구하시면 됩니다.

In [ ]:
# trasform 함수 사용 X 방법

# M세대로 나누기
df_m = df[(df["generation"] == "M")]

# 세션아이디별로 타임스탬프 정렬
df = df_m.sort_values(by=['session_id', 'event_timestamp'])

# 마지막 타임스탬프만 뽑기
last_timestamp = df.groupby(by="session_id")["event_timestamp"].max().reset_index()

# 마지막 타임스탬프와 첫번째 타임스탬프 합치기
result = pd.merge(df, last_timestamp, on = "session_id")

# 첫번째 세션 아이디만 남기고 중복된 세션아이디 제거
result = result.drop_duplicates(subset=["session_id"])

# 문자열을 날짜 데이터타입으로 변환 
result['event_timestamp_x'] = pd.to_datetime(result['event_timestamp_x'])
result['event_timestamp_y'] = pd.to_datetime(result['event_timestamp_y'])

# 세션별 최대 타임과 최소 타임의 차이 구하기
time = result['event_timestamp_y'] - result['event_timestamp_x']


# 단순 세션별 
time.sum()/(df['session_id'].drop_duplicates().count())
# > Timedelta('0 days 00:05:24.772675586')
#세션당 평균 체류시간 

# 세션별 - 차이가 0인 세션 제외한 세션수로 나누고
# 액티브한 고객들의 세션만 이용하면 높은 수가 나올 줄 알았지만, 아니었음
time.sum()/time[time > '00:00:00'].shape[0]
# > Timedelta('0 days 00:08:25.438678803')

# time.sum()/555908 # time[time > '00:00:00'] 만족하는 행의 수로 나누기

In [9]:
# M세대로 나누기
df_m = df[(df["generation"] == "M")]

In [10]:
# 세션아이디별로 타임스탬프 정렬
df = df_m.sort_values(by=['session_id', 'event_timestamp'])

In [11]:
# 마지막 타임스탬프만 뽑기
last_timestamp = df.groupby(by="session_id")["event_timestamp"].max().reset_index()

In [12]:
# 마지막 타임스탬프와 첫번째 타임스탬프 합치기
result = pd.merge(df, last_timestamp, on = "session_id")

In [13]:
# 첫번째 세션 아이디만 남기고 중복된 세션아이디 제거
result = result.drop_duplicates(subset=["session_id"])

In [14]:
# 문자열을 날짜 데이터타입으로 변환 
result['event_timestamp_x'] = pd.to_datetime(result['event_timestamp_x'])
result['event_timestamp_y'] = pd.to_datetime(result['event_timestamp_y'])

In [15]:
# 세션별 최대 타임과 최소 타임의 차이 구하기
time = result['event_timestamp_y'] - result['event_timestamp_x']

In [17]:
# 단순 세션별 
time.sum()/(df['session_id'].drop_duplicates().count())

Timedelta('0 days 00:05:24.772675586')

In [18]:
# 세션별 - 차이가 0인 세션 제외한 세션수로 나누고
# 액티브한 고객들의 세션만 이용하면 높은 수가 나올 줄 알았지만, 아니었음
time.sum()/time[time > '00:00:00'].shape[0]

Timedelta('0 days 00:08:25.438678803')

## 세션 방문 빈도수

In [ ]:
# session_id 을 기준으로 중복 제거 
df_session = df.drop_duplicates(['session_id'])
df_session

# 세대별로 그룹화
grouped_mz = df_session.groupby('generation')

# M세대 user_no 별로 groupby 
grouped_m_user_no= grouped_mz.get_group('M').groupby('user_no')
grouped_m_user_no.size().sum() # 865151
# Z세대 user_no 별로 groupby
grouped_z_user_no= grouped_mz.get_group('Z').groupby('user_no')
grouped_z_user_no.size().sum() # 319240

# M세대의 세션 빈도수의 평균
grouped_m_user_no.size().mean() # 6.013296448951506
# Z세대의 세션 빈도수의 평균
grouped_z_user_no.size().mean() # 5.153687201343149

# M세대의 세션 빈도수의 중위수
grouped_m_user_no.size().median() # 3.0
# Z세대의 세션 빈도수의 중위수
grouped_z_user_no.size().median() # 2.0

In [19]:
# session_id 을 기준으로 중복 제거 
df_session = df.drop_duplicates(['session_id'])
df_session

,user_no,year,generation,session_id,event_timestamp,event_name,item_no,price,category1_name,category2_name,category3_name,brand_no,brand_name
1774282,a+qjV4Mq5z4UwP69U7B8aw==,1982,M,0000104cd168386a335ba6bf6e32219d,2021-07-20 02:16:04.405017,add_to_cart,9W/h68eEr+AVlH3aQnvpSA==,179000.0,잡화,신발,샌들,10219.0,CHUBASCO
1180948,V4z7pgAxLNqN5YpcT19ocA==,1982,M,0000193a728fd00b6cff91b8840bbf8d,2021-06-10 01:36:21.357033,click_item,Wv+sVIZvHu8e2sRfYmYgYQ==,37000.0,스포츠/레저,캠핑,기타 캠핑용품,1669.0,HIBROW company
2560870,+Ah1Nbgq/LVp6XX7uhlaoQ==,1992,M,00003e3b9e5336685200ae85d21b4f5e,2021-08-02 07:22:06.236029,click_item,cBuYkZ0BHhrAw3APdaZXsw==,39000.0,의류,상의,티셔츠,6129.0,ARETHAT
1967466,vPnWvRSiCVhmzoyVC3AjQQ==,1987,M,000051f32d02c8a53b43a686969676e6,2021-07-21 11:08:40.341108,click_item,2CE69ahbX9niENVTr9u7yQ==,93000.0,의류,상의,카디건,2069.0,Andersson Bell for women
2219248,MrV6uhZQS4D/EzBii/JBVg==,1989,M,00007c55a9a7591b98a76d79216c9112,2021-07-03 23:15:04.800098,click_item,dWDUiV7Gd2bkvwfJXR7iOQ==,33500.0,잡화,가방,에코백,10693.0,민민
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134683,Pi7/bxomllN+TNE2XnlhMw==,1991,M,ffff9979c9699b51cb7cda98e5bf84c2,2021-07-15 18:11:18.478039,click_item,vR8cgga99YIhifEb//ffDw==,59000.0,의류,상의,니트/스웨터,2003.0,AMONG
136319,8TaE3hhkfhn8TtrPayhZIA==,1993,M,ffffb8de2d77567f452ad766471a9e5e,2021-06-21 07:49:53.120004,purchase_success,ZaLANVgSFf9opvMKO9zgLw==,12000.0,컬처,티켓,전시/행사,14747.0,그라운드시소
1587021,Ej6vc1bpQU3E78ZJGrM8xg==,1994,M,ffffbd0f062adb4c4a307c5a05f7b0ad,2021-07-13 21:55:53.220088,click_item,Hq+1p/y1/AXj8rr4Tk9IUg==,99000.0,의류,상의,티셔츠,13227.0,POLO RALPH LAUREN
2592175,2Dv1XGrHGK5/8ORRBHjF6Q==,1992,M,ffffda8ada4827900cbf65fd20281080,2021-07-01 09:06:01.725,click_item,BpEE6ZLim9KZNlpCUNTWiw==,136000.0,의류,스윔웨어,원피스수영복,5892.0,JULIY&CALLA


In [20]:
# 세대별로 그룹화
grouped_mz = df_session.groupby('generation')

In [21]:
# M세대 user_no 별로 groupby 
grouped_m_user_no= grouped_mz.get_group('M').groupby('user_no')
grouped_m_user_no.size().sum()

865151

In [22]:
# M세대의 세션 빈도수의 평균
grouped_m_user_no.size().mean()

6.013296448951506

In [23]:
# M세대의 세션 빈도수의 중위수
grouped_m_user_no.size().median() #3.0

3.0

> 여기까지 쭉 따라하다가 Z세대 구하려고 하면 아래와 같이 오류가 뜰텐데요,  
> 지표 1.1, 1.2 구하면서 df의 generation = "M" 상태로 바껴서 안되는 것 같습니다.  
> Z세대 구하시려면 다시 파일 불러오셔서 하시는 걸 추천드려요!

In [24]:
# Z세대 user_no 별로 groupby
grouped_z_user_no= grouped_mz.get_group('Z').groupby('user_no')
grouped_z_user_no.size()

KeyError: 'Z'

In [60]:
# Z세대의 세션 빈도수의 평균
grouped_z_user_no.size().mean() 

5.153687201343149

In [61]:
# Z세대의 세션 빈도수의 중위수
grouped_z_user_no.size().median() 

2.0